In [1]:
import os
import diff_classifier.imagej as ij
import boto3
import os.path as op
import diff_classifier.aws as aws

In [ ]:
import cloudknot as ck

In [19]:
def download_and_track(filename):
    
    import diff_classifier.imagej as ij
    import diff_classifier.aws as aws
    import os.path as op
    
    aws.download_s3(filename, op.split(filename)[1])
    
    outfile = 'Traj_' + op.split(filename)[1].split('.')[0] + '.csv'

    if not op.isfile(outfile):
        ij.track(op.split(filename)[1], outfile, template=None, fiji_bin=None, radius=3.5, threshold=0., 
              do_median_filtering=True, quality=30.0, median_intensity=300.0, snr=0.0, 
              linking_max_distance=7.0, gap_closing_max_distance=10.0, max_frame_gap=2,
              track_displacement=10.0)

        aws.upload_s3(outfile, op.join(op.split(filename)[0], outfile))

In [20]:
download_and_track('01_18_Experiment/P1_S1_L_0000_0_0.tif')

In [101]:
def download_and_split(filename):
    
    import diff_classifier.imagej as ij
    import diff_classifier.aws as aws
    import os.path as op
    
    local_name = op.split(filename)[1]
    DIR = op.split(filename)[0]
    try1 = filename.split('.')[0] + '_0_0.tif'
    try2 = filename.split('.')[0] + '_3_3.tif'
    
    s3 = boto3.client('s3')
    try:
        obj = s3.head_object(Bucket='ccurtis7.pup', Key=try1)
    except:
        try:
            obj = s3.head_object(Bucket='ccurtis7.pup', Key=try2)
        except:
            aws.download_s3(filename, local_name)
            names = ij.partition_im(local_name)
            for name in names:
                aws.upload_s3(name, op.join(op.split(filename)[0], name))

In [34]:
pups = ["P1", "P2", "P3"]
slices = ["S1", "S2", "S3"]
hemis = ["R", "L"]
videos = 15
folder = '01_18_Experiment'

In [102]:
missing = []
for i in range(10, 15):
    missing.append("P1_S2_R_00{}.tif".format(i))

for i in range(10, 15):
    missing.append("P2_S3_L_00{}.tif".format(i))
    
for i in range(0, 15):
    missing.append("P3_S3_L_{}.tif".format("%04d" % i))

In [103]:
to_partition = []
for pup in pups:
    for slic in slices:
        for hemi in hemis:
            for i in range(0, videos):
                name = "{}/{}/{}_{}_{}_{}.tif".format(folder, pup, pup, slic, hemi, "%04d" % i)
                if not name in missing:
                    to_partition.append(name)

In [105]:
for name in to_partition:
    download_and_split(name)

/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0006_0_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0006_0_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0006_0_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0006_0_3.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0006_1_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0006_1_1.tif i

/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0008_3_3.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0009_0_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0009_0_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0009_0_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0009_0_3.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0009_1_0.tif i

/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0011_3_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0011_3_3.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0012_0_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0012_0_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0012_0_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0012_0_3.tif i

/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0014_3_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0014_3_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: P1_S1_L_0014_3_3.tif is a low contrast image
  warn('%s is a low contrast image' % fname)


ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [51]:
!rm P1_S1_R_0000*

In [ ]:
to_partition

In [55]:
import boto3

In [71]:
s3 = boto3.resource('s3')

try:
    s3.Object("ccurtis7.pup", '01_18_Experiment/test_file.tif').load()

'P1_S1_R_0000_0_0.tif'

In [92]:
s3 = boto3.client('s3')

result = s3.list_objects_v2(Bucket="ccurtis7.pup", Prefix="/01_18_Experiment/test_filebook.tif")

exists = False
if result:
    exists = True

print(exists)

True


In [93]:
for obj in result.get('Contents', []):
    if obj['Key'] == "/01_18_Experiment/test_filebook.tif":
        print('Object here')
    else:
        print('Object not here')

In [100]:
try:
    obj = s3.head_object(Bucket='ccurtis7.pup', Key="01_18_Experiment/test_file.tif")
except:
    print("Does not exist")